### Импорт библиотек

In [69]:
from itertools import combinations
from sklearn.preprocessing import OrdinalEncoder
from tqdm import tqdm
from bs4 import BeautifulSoup
import urllib.request, sys, time
import requests
from razdel import sentenize
import pandas as pd
import re
import numpy as np
from bs4 import BeautifulSoup
import contractions
import re
import tqdm
import unicodedata
from random import seed
seed(2)
import pymorphy2
import ast

import syllables
import rusyllab
import seaborn as sns
%matplotlib inline

from nltk import sent_tokenize, word_tokenize
import holidays

import nltk
from nltk.corpus import stopwords
stopwords = set(stopwords.words('russian'))
nltk.download('wordnet')
nltk.download('stopwords');
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score

from tqdm.contrib import tzip
from time import sleep
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

from pymystem3 import Mystem
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
tqdm.pandas() 

ru_holidays = holidays.country_holidays('RU')
from collections import Counter

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\steam\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\steam\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\Users\steam\anaconda3\lib\site-packages\tqdm\std.py:670: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
df_train = pd.read_csv('full_last_train.csv').drop(columns=['Unnamed: 0'])

### Удаление выбросов

In [3]:
df_train = df_train[df_train['full_reads_percent']<100]

In [65]:
df_train.head(2)

,views,full_reads_percent,depth,authors,cat,ctr,day,month,hour,weekend,...,tag_КНДР,tag_Виктор Орбан,tag_посол,tag_рубли,tag_главные новости,tag_пожары,tag_Джо Байден,tag_самолет,tag_Евросоюз,tag_Дети
0,20460,35.850,1.134,560,Финансы,1.580,4,4,10,0,...,0,0,0,0,0,0,0,0,0,0
1,19038,38.355,1.142,38,Политика,1.853,18,2,10,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df_test = pd.read_csv('full_last_test.csv').drop(columns=['Unnamed: 0'])

### Получение информации о тэгах

In [6]:
def get_all_tags(df_tags):
    tags = []
    for tag in df_tags:
        t = ast.literal_eval(tag)
        tags.extend(t)
    return tags

In [7]:
def get_tags_info(df):
    tags, views, fulls = [], [], []
    for tag, view, full in zip(df['tag'], df['views'], df['full_reads_percent']):
        t = ast.literal_eval(tag)
        v = [view]*len(t)
        f = [full]*len(t)
        tags.extend(t)
        views.extend(v)
        fulls.extend(f)
    return tags, views, fulls

In [8]:
def get_most_common(all_tags, num=350):
    most_common = Counter(all_tags).most_common()[:num]
    most_common = [common[0] for common in most_common]
    return most_common

In [9]:
def get_binary_tags(tags):
    binary_tags = np.zeros(len(most_common_tags), dtype=int)
    tags = ast.literal_eval(tags)
    for tag in tags:
        try:
            index = most_common_tags.index(tag)
            binary_tags[index] = 1
        except:
            pass
    return list(binary_tags)

In [10]:
tags, views, fulls = get_tags_info(df_train)

In [11]:
tag_dic = dict(Counter(tags))

In [12]:
voc = {'tags':tags, 'views':views, 'full':fulls}
tag_info = pd.DataFrame(voc)

In [13]:
def count_tag(tag):
    return tag_dic[tag]

In [14]:
col = 'views'
views_tags = pd.DataFrame(tag_info.groupby('tags')[col].mean())
views_tags = views_tags.sort_values(by=[col], ascending=False).reset_index()

In [15]:
views_tags['count_tag'] = views_tags['tags'].apply(count_tag)

### Получение влияющих на просмотры тэгов

In [16]:
views_tags[views_tags['count_tag'] > 20]['tags'][:20]

41                    рубль
43          главные новости
61               Джо Байден
73                    рубли
150                    НАТО
165                 Газпром
230                     газ
241                  Россия
251          Владимир Путин
272                  Сербия
328          Рамзан Кадыров
357    Владимир Жириновский
388                     США
415                     ЛНР
422                 экспорт
451                   Чечня
473                     ДНР
474              Олаф Шольц
480                Евросоюз
492                    Киев
Name: tags, dtype: object

In [17]:
view_tags = list(views_tags[views_tags['count_tag'] > 20]['tags'][:20])

In [18]:
col = 'full'
views_tags = pd.DataFrame(tag_info.groupby('tags')[col].mean())
views_tags = views_tags.sort_values(by=[col], ascending=False).reset_index()
views_tags['count_tag'] = views_tags['tags'].apply(count_tag)

### Получение тэгов, влияющих на процент полного просмотра

In [19]:
views_tags[views_tags['count_tag'] > 20]['tags'][:20]

409              мультимедиа
677                    Пожар
767                     КНДР
915                   пожары
1022                 самолет
1058                 встреча
1091                стрельба
1137                  Погода
1225                     ДТП
1232                    Дети
1306                  учения
1332    Следственный комитет
1346                      СК
1381            Виктор Орбан
1386     Александр Бастрыкин
1423                 обстрел
1454                  Япония
1457                     МЧС
1473    Владимир Жириновский
1486                   посол
Name: tags, dtype: object

In [20]:
full_tags = list(views_tags[views_tags['count_tag'] > 20]['tags'][:20])

### Переводим тэги в бинарное представление

In [1]:
def df_binary_tags(df_train):
    all_binary = []
    for tags in df_train['tag']:
        binary_tags = list(get_binary_tags(tags))
        all_binary.append(binary_tags)
    tag_names = ['tag_'+tag for tag in most_common_tags]
    result = np.array(all_binary, dtype=int)
    df_tags = pd.DataFrame(result, columns=tag_names, index=df_train.index)
    return df_tags

In [22]:
final_tags = view_tags + full_tags

In [23]:
most_common_tags = list(set(final_tags))

In [24]:
train_tags = df_binary_tags(df_train)

In [71]:
train_tags.head(1)

,tag_НАТО,tag_Рамзан Кадыров,tag_ЛНР,tag_СК,tag_Япония,tag_Следственный комитет,tag_Газпром,tag_Владимир Путин,tag_рубль,tag_Олаф Шольц,...,tag_КНДР,tag_Виктор Орбан,tag_посол,tag_рубли,tag_главные новости,tag_пожары,tag_Джо Байден,tag_самолет,tag_Евросоюз,tag_Дети
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
test_tags = df_binary_tags(df_test)

In [70]:
test_tags.head(1)

,tag_НАТО,tag_Рамзан Кадыров,tag_ЛНР,tag_СК,tag_Япония,tag_Следственный комитет,tag_Газпром,tag_Владимир Путин,tag_рубль,tag_Олаф Шольц,...,tag_КНДР,tag_Виктор Орбан,tag_посол,tag_рубли,tag_главные новости,tag_пожары,tag_Джо Байден,tag_самолет,tag_Евросоюз,tag_Дети
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
columns_ = pd.DataFrame(columns=train_tags.columns)

In [59]:
df_train.head(2)

,views,full_reads_percent,depth,authors,cat,ctr,day,month,hour,weekend,...,tag_КНДР,tag_Виктор Орбан,tag_посол,tag_рубли,tag_главные новости,tag_пожары,tag_Джо Байден,tag_самолет,tag_Евросоюз,tag_Дети
0,20460,35.850,1.134,560,Финансы,1.580,4,4,10,0,...,0,0,0,0,0,0,0,0,0,0
1,19038,38.355,1.142,38,Политика,1.853,18,2,10,0,...,0,0,0,0,0,0,0,0,0,0


### Объединяем тестовый и трейновый датасет с тэгами

In [32]:
df_train = pd.concat([df_train, train_tags], axis= 1).drop(columns=['tag'])

In [36]:
df_test = pd.concat([df_test, test_tags], axis= 1).drop(columns=['tag'])

In [37]:
print(len(df_test))
df_test.head(1)

3000


,authors,cat,ctr,day,month,hour,weekend,holiday,full_affect,view_affect,...,tag_КНДР,tag_Виктор Орбан,tag_посол,tag_рубли,tag_главные новости,tag_пожары,tag_Джо Байден,tag_самолет,tag_Евросоюз,tag_Дети
0,560,Военная операция на Украине,0.0,1,2,17,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Делим датасет на признаки и целевую переменную

In [38]:
X = df_train.drop(columns=['views', 'full_reads_percent','depth'])
y_full = df_train['full_reads_percent']
y_views = df_train['views']
y_depth = df_train['depth']
Y = [y_full, y_views, y_depth]

### Ввыбираем категориальные колонки для OHE

In [42]:
cat_cols = [0,1,3,4,5,224]

### Высчитываем размерность всего OHE

In [63]:
ohe_cols = 0
for col in cat_cols:
    ohe_cols += len(X[X.columns[col]].unique())

### Предсказание таргетов

In [47]:
model = CatBoostRegressor(random_state=0, verbose=0, one_hot_max_size=ohe_cols)
model.fit(X, y_views, cat_features=cat_cols)
views = model.predict(df_test)

In [51]:
model = CatBoostRegressor(random_state=0, verbose=0, one_hot_max_size=ohe_cols)
model.fit(X, y_full, cat_features=cat_cols)
full = model.predict(df_test)

In [53]:
model = CatBoostRegressor(random_state=0, verbose=0, one_hot_max_size=ohe_cols)
model.fit(X, y_depth, cat_features=cat_cols)
depth = model.predict(df_test)

### Сохранение в файл

In [55]:
test = pd.read_csv('test_dataset_test.csv', delimiter=',')

voc = {'document_id': test['document_id'].values, 'views':views , 'depth':depth, 'full_reads_percent':full}
res_df = pd.DataFrame(voc)
res_df = res_df.set_index('document_id')
res_df.to_csv('categories_catboost_ohe_tags_40_full_views_tags.csv')